# **Aircraft Purchase:**

Problem Source: F. S. Hillier and G. J. Lieberman, "Introduction to Mathematical Programming," 2nd Edition, McGraw-Hill Inc., New York, 1995.

**Problem Statement:**

An airline company is considering the purchase of new long-, medium-, and short-range jet passenger airplanes. The purchase price would be $ \$33.5$ million for each longrange plane, $ \$25$ million for each medium-range plane, and $ \$17.5$ million for each short-range plane. The board of directors has authorized a maximum commitment of $ \$750$ million for these purchases. Regardless of which airplanes are purchased, air travel of all distances is expected to
be sufficiently large that these planes would be utilized at essentially maximum capacity. It is estimated that the net annual profit (after capital recovery costs are subtracted) would be $ \$2.1$ million per long-range plane, $ \$1.5$ million per medium-range plane, and $ \$1.15$ million per shortrange plane.

It is predicted that enough trained pilots will be available to the company to crew $30$ new airplanes. If only short-range planes were purchased, the maintenance facilities would be able to handle $40$ new planes. However, each medium-range plane is equivalent to $1 \frac{1}{3}$ short-range planes, and each long-range plane is equivalent to $1 \frac{2}{3}$ short-range planes in terms of their use of the maintenance facilities.

The information given here was obtained by a preliminary analysis of the problem. A more detailed analysis will be conducted subsequently. However, using the preceding data as a first approximation, management wishes to know how many planes of each type should be purchased to maximize profit.

**Modeling:**

$x_{i} \qquad  i\in {1,2,3} \quad $   integer variables for number of planes purchased

$\\ $

$Objective\ Function:\\ $

$Max\ Z = 2.1x_{1}+1.5x_{2}+1.15x_{3}$

$\\ $

$Constraints:\\ $

Budget Constraint:

$33.5x_{1}+25x_{2}+17.5x_{3} \leq 750$

Number of Planes Constraint:

$x_{1}+x_{2}+x_{3} \leq 30$

Maintenance Facilities Constraint:

$1\frac{2}{3}x_{1}+1\frac{1}{3}x_{2}+x_{3} \leq 40$

Binary Constraints:

$x_{1},x_{2},x_{3} \in {0,1}$

## Using Pulp Library

In [1]:
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd

In [2]:
#Defining Variables

x = LpVariable.dicts('x', ((i) for i in range(1,4)), lowBound=0, cat='Integer')

In [3]:
#Defining Objective Function

model = LpProblem('model', LpMaximize)

model += 2.1*x[1]+1.5*x[2]+1.15*x[3]

In [4]:
#Constraints

#Budget constraint
model += 33.5*x[1]+25*x[2]+17.5*x[3] <= 750

#Number of planes constraint
model += x[1]+x[2]+x[3] <= 30

#Maintenance facilities constraint:
model += (5/3)*x[1]+(4/3)*x[2]+x[3] <= 40

In [5]:
#Model summary

model

model:
MAXIMIZE
2.1*x_1 + 1.5*x_2 + 1.15*x_3 + 0.0
SUBJECT TO
_C1: 33.5 x_1 + 25 x_2 + 17.5 x_3 <= 750

_C2: x_1 + x_2 + x_3 <= 30

_C3: 1.66666666667 x_1 + 1.33333333333 x_2 + x_3 <= 40

VARIABLES
0 <= x_1 Integer
0 <= x_2 Integer
0 <= x_3 Integer

In [6]:
#Solving model

model.solve()
print('Optimal Solution:', pulp.value(model.objective))

Optimal Solution: 47.8


In [7]:
for variables in model.variables():
  print(f'{variables.name}:{variables.varValue}')

x_1:14.0
x_2:0.0
x_3:16.0


In [8]:
VNames = []
for variables in model.variables():
  VNames.append(variables.name)

VValue = []
for variables in model.variables():
  VValue.append(np.round(variables.varValue, 2))

data = {'Variables': VNames, 'Value': VValue}
pd.DataFrame(data, index = range(1, len(VValue)+1))

,Variables,Value
1,x_1,14.0
2,x_2,0.0
3,x_3,16.0


In [9]:
print('Current Status: ', LpStatus[model.status])

Current Status:  Optimal


## Using Pyomo Library

In [10]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

from pyomo.environ import *

In [11]:
model = ConcreteModel()

model.indices = Set(initialize=list(range(1,4)))
model.x = Var(model.indices, bounds=(0, None), domain=Integers)

model.obj = Objective(expr=2.1*model.x[1]+1.5*model.x[2]+1.15*model.x[3], sense=maximize)

model.constraint1 = Constraint(expr=33.5*model.x[1]+25*model.x[2]+17.5*model.x[3] <= 750)
model.constraint2 = Constraint(expr=model.x[1]+model.x[2]+model.x[3] <= 30)
model.constraint3 = Constraint(expr=(5/3)*model.x[1]+(4/3)*model.x[2]+model.x[3] <= 40)

SolverFactory('glpk').solve(model)

model.display()

Model unknown

  Variables:
    x : Size=3, Index=indices
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  14.0 :  None : False : False : Integers
          2 :     0 :   0.0 :  None : False : False : Integers
          3 :     0 :  16.0 :  None : False : False : Integers

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  47.8

  Constraints:
    constraint1 : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 749.0 : 750.0
    constraint2 : Size=1
        Key  : Lower : Body : Upper
        None :  None : 30.0 :  30.0
    constraint3 : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 39.333333333333336 :  40.0


In [12]:
print("Variable values:")
model.x.pprint()

Variable values:
x : Size=3, Index=indices
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  14.0 :  None : False : False : Integers
      2 :     0 :   0.0 :  None : False : False : Integers
      3 :     0 :  16.0 :  None : False : False : Integers
